# CII proyect: Data Preparation
### First data preparation process, from raw data.
### Data generated:
- Enviromental Data
    - Dumps
- Urban Data
    - RENABAP
    - Green Space
    - Health Buldings
    - Education Buildings
    - Sport and community centers
    - Roads and transport
    - Shops
    - Land Use
    - Security
- Socialdemographic Data
    - Census Data

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# import local goverments
df_muni = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/municipio/municipio.shp")
df_muni = df_muni.loc[df_muni.nam == 'La Matanza']

# Enviromental Data

### Dumps

In [3]:
# import data
df_bas = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Ambiental/VertederosBasureros.geojson")
df_rel = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Ambiental/RellenoSanitario.geojson")

# concat dfs
df_basca = pd.concat([df_bas, df_rel])

# dumps in La Matanza
df_basca_lm = df_basca.sjoin(df_muni[['geometry']])

# save layer
df_basca_lm.drop(columns='index_right', inplace=True)
df_basca_lm.to_file("../DatosProcesados/Ambientales/BasuralesACieloAbierto.geojson", driver='GeoJSON')

# Urban Data

### RENABAP

In [4]:
# import data
df_renabap = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/RENABAP_2022-07-13_info_publica.geojson")

# dumps in La Matanza
df_renabap_lm = df_renabap.sjoin(df_muni[['geometry']])

# save layer
df_renabap_lm.drop(columns='index_right', inplace=True)
df_renabap_lm.to_file("../DatosProcesados/Urbanos/BarriosRENABAP.geojson", driver='GeoJSON')

### Green Space

In [5]:
# import data
df_evp_ign = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/espacios verdes/EspaciosVerdes.geojson")
df_evp_osm = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/espacios verdes/leisure_park_pol.geojson")
df_rn_osm = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/espacios verdes/leisure_naturereserve_pol.geojson")

# filter OSM data not overlaying IGN data 
df_evp_osm_sj = df_evp_osm.sjoin(df_evp_ign[['gid', 'geometry']], how='left')
df_evp_osm_sj = df_evp_osm_sj.loc[df_evp_osm_sj.gid.isnull()]
# clean data
df_evp_osm_sj.drop(columns='gid', inplace=True)
df_evp_osm_sj.rename(columns={'full_id':'gid', 'name':'nam'}, inplace=True)
df_evp_osm_sj['sag'] = 'OSM'
df_evp_osm_sj['objeto'] = 'Espacio Verde'

# filter OSM data not overlaying IGN data 
df_rn_osm_sj = df_rn_osm.sjoin(df_evp_ign[['gid', 'geometry']], how='left')
df_rn_osm_sj = df_rn_osm_sj.loc[df_rn_osm_sj.gid.isnull()]
# clean data
df_rn_osm_sj.drop(columns='gid', inplace=True)
df_rn_osm_sj.rename(columns={'full_id':'gid', 'name':'nam'}, inplace=True)
df_rn_osm_sj['sag'] = 'OSM'
df_rn_osm_sj['objeto'] = 'Reserva Natural'

# concat dfs
df_epv = pd.concat([df_evp_ign, df_evp_osm_sj, df_rn_osm_sj]).reset_index(drop=True)
df_epv = df_epv[['gid', 'objeto', 'sag', 'geometry']]
df_epv

# epvs in La Matanza
df_epv_lm = df_epv.sjoin(df_muni[['geometry']])
# calculate surface and density
df_epv_lm = df_epv_lm.to_crs("EPSG:5347")
df_epv_lm['sup_m2'] = df_epv_lm.area
df_epv_lm = df_epv_lm.to_crs("EPSG:4326")
# save layer
df_epv_lm.drop(columns='index_right', inplace=True)
df_epv_lm.to_file("../DatosProcesados/Urbanos/EspaciosVerdes.geojson", driver='GeoJSON')

### Health Buldings

In [6]:
# import data
df_salud = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/establecimientos_salud_publicos/Establecimientos_salud_publicos_alta_al20220721.shp")

# filter fields and split data
df_salud = df_salud[['gna', 'nam', 'cat', 'geometry']]
df_hosp = df_salud.loc[df_salud.cat == 'Hospital']
df_caps = df_salud.loc[(df_salud.cat == 'C.A.P.S') | (df_salud.cat == 'C.I.C.') | (df_salud.cat == 'C.A.P.S de instituciones cerradas') | (df_salud.cat == 'Unidad Móvil')
             | (df_salud.cat == 'U.P.A.') | (df_salud.cat == 'Centro de día') | (df_salud.cat == 'Emergencias') | (df_salud.cat == 'Posta Sanitaria')]

# hospitals and health centers in La Matanza
df_hosp_lm = df_hosp.sjoin(df_muni[['geometry']])
df_caps_lm = df_caps.sjoin(df_muni[['geometry']])

# save layer
df_hosp_lm.drop(columns='index_right', inplace=True)
df_caps_lm.drop(columns='index_right', inplace=True)
df_hosp_lm.to_file("../DatosProcesados/Urbanos/Hospitales.geojson", driver='GeoJSON')
df_caps_lm.to_file("../DatosProcesados/Urbanos/CentrosDeAtencion_Salud.geojson", driver='GeoJSON')

### Education Buildings

In [12]:
# import data
df_edu = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/establecimientos_educativos/shapefile/establecimientos_educativos.shp")
# filter fields
df_edu = df_edu[['nombre', 'sector', 'nivel', 'tipo_organ', 'geometry']]

# split data by sector
df_edu_pub = df_edu.loc[(df_edu.sector == 'Estatal')]
df_edu_pri = df_edu.loc[(df_edu.sector == 'Privado')]
# split data by level
df_edu_pub_ini = df_edu_pub.loc[(df_edu_pub.nivel == 'Ciclo de Iniciación') | (df_edu_pub.nivel == 'Nivel Inicial')]
df_edu_pub_pri = df_edu_pub.loc[(df_edu_pub.nivel == 'Nivel Primario')]
df_edu_pub_sec = df_edu_pub.loc[(df_edu_pub.nivel == 'Ciclo Medio') | (df_edu_pub.nivel == 'Nivel Superior (**)') | (df_edu_pub.nivel == 'Nivel Superior (**)')| (df_edu_pub.nivel == 'Nivel Secundario')
| (df_edu_pub.nivel == 'Nivel Medio')]
df_edu_pub_ter = df_edu_pub.loc[(df_edu_pub.nivel == 'Cursos y Talleres') | (df_edu_pub.nivel == 'Formación Integral') | (df_edu_pub.nivel == 'Residencia Laboral,Pasantias, Artística')
 | (df_edu_pub.nivel == 'Formación Profesional')| (df_edu_pub.nivel == 'Educación Física (C.E.F.)')]
df_edu_pri_ini = df_edu_pri.loc[(df_edu_pri.nivel == 'Ciclo de Iniciación') | (df_edu_pri.nivel == 'Nivel Inicial')]
df_edu_pri_pri = df_edu_pri.loc[(df_edu_pri.nivel == 'Nivel Primario')]
df_edu_pri_sec = df_edu_pri.loc[(df_edu_pri.nivel == 'Ciclo Medio') | (df_edu_pri.nivel == 'Nivel Superior (**)') | (df_edu_pri.nivel == 'Nivel Superior (**)')| (df_edu_pri.nivel == 'Nivel Secundario')
| (df_edu_pri.nivel == 'Nivel Medio')]
df_edu_pri_ter = df_edu_pri.loc[(df_edu_pri.nivel == 'Cursos y Talleres') | (df_edu_pri.nivel == 'Formación Integral') | (df_edu_pri.nivel == 'Residencia Laboral,Pasantias, Artística')
 | (df_edu_pri.nivel == 'Formación Profesional')| (df_edu_pri.nivel == 'Educación Física (C.E.F.)')]

# buildings in La Matanza
df_edu_pub_ini_lm = df_edu_pub_ini.sjoin(df_muni[['geometry']])
df_edu_pub_pri_lm = df_edu_pub_pri.sjoin(df_muni[['geometry']])
df_edu_pub_sec_lm = df_edu_pub_sec.sjoin(df_muni[['geometry']])
df_edu_pub_ter_lm = df_edu_pub_ter.sjoin(df_muni[['geometry']])
df_edu_pri_ini_lm = df_edu_pri_ini.sjoin(df_muni[['geometry']])
df_edu_pri_pri_lm = df_edu_pri_pri.sjoin(df_muni[['geometry']])
df_edu_pri_sec_lm = df_edu_pri_sec.sjoin(df_muni[['geometry']])
df_edu_pri_ter_lm = df_edu_pri_ter.sjoin(df_muni[['geometry']])

# save layer
df_edu_pub_ini_lm.drop(columns='index_right', inplace=True)
df_edu_pub_pri_lm.drop(columns='index_right', inplace=True)
df_edu_pub_sec_lm.drop(columns='index_right', inplace=True)
df_edu_pub_ter_lm.drop(columns='index_right', inplace=True)
df_edu_pri_ini_lm.drop(columns='index_right', inplace=True)
df_edu_pri_pri_lm.drop(columns='index_right', inplace=True)
df_edu_pri_sec_lm.drop(columns='index_right', inplace=True)
df_edu_pri_ter_lm.drop(columns='index_right', inplace=True)
df_edu_pub_ini_lm.to_file("../DatosProcesados/Urbanos/EstEducativo_Publico_Inicial.geojson", driver='GeoJSON')
df_edu_pub_pri_lm.to_file("../DatosProcesados/Urbanos/EstEducativo_Publico_Primaria.geojson", driver='GeoJSON')
df_edu_pub_sec_lm.to_file("../DatosProcesados/Urbanos/EstEducativo_Publico_Secundaria.geojson", driver='GeoJSON')
df_edu_pub_ter_lm.to_file("../DatosProcesados/Urbanos/EstEducativo_Publico_Terciaria.geojson", driver='GeoJSON')
df_edu_pri_ini_lm.to_file("../DatosProcesados/Urbanos/EstEducativo_Privado_Inicial.geojson", driver='GeoJSON')
df_edu_pri_pri_lm.to_file("../DatosProcesados/Urbanos/EstEducativo_Privado_Primaria.geojson", driver='GeoJSON')
df_edu_pri_sec_lm.to_file("../DatosProcesados/Urbanos/EstEducativo_Privado_Secundaria.geojson", driver='GeoJSON')
df_edu_pri_ter_lm.to_file("../DatosProcesados/Urbanos/EstEducativo_Privado_Terciaria.geojson", driver='GeoJSON')

### Sport and community centers

In [7]:
# import data
df_sport = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/OSM_data/OSM_sport.geojson")
df_sport_pol = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/OSM_data/OSM_sport_pol.geojson")
df_comcenter = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/OSM_data/OSM_amenity_community_center.geojson")
# change df_sport_pol geometry
df_sport_pol.geometry = df_sport_pol.centroid
df_sport = pd.concat([df_sport, df_sport_pol]).reset_index(drop=True)
# filter fields
df_sport = df_sport[['full_id', 'name', 'sport', 'leisure', 'geometry']]
df_comcenter = df_comcenter[['full_id', 'name', 'club', 'community_centre', 'geometry']]

# buildings in La Matanza
df_sport_lm = df_sport.sjoin(df_muni[['geometry']])
df_comcenter_lm = df_comcenter.sjoin(df_muni[['geometry']])

# save layer
df_sport_lm.drop(columns='index_right', inplace=True)
df_comcenter_lm.drop(columns='index_right', inplace=True)
df_sport_lm.to_file("../DatosProcesados/Urbanos/CentrosDeportes.geojson", driver='GeoJSON')
df_comcenter_lm.to_file("../DatosProcesados/Urbanos/CentrosComunitarios.geojson", driver='GeoJSON')

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


### Roads and transport

In [14]:
# import data
df_redvial = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/red-vial/red-vial.shp")
df_redcol = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/recorrido-colectivo-de-lineas-provinciales/shapefile/recorrido-colectivo-de-lineas-provinciales.shp")
df_estcol = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/route_busstop_point.geojson")
df_redffcc = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/ferrocarril-provincia/ferrocarril-provincia.shp")
df_estffcc = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/estaciones-ferrocarril-provincia/estaciones-ferrocaril-provincia.shp")
# filter fields
df_redvial = df_redvial.loc[(df_redvial.fna1.notnull()) | (df_redvial.nr.notnull())]

# data in La Matanza
df_redvial_lm = gpd.overlay(df_redvial, df_muni[['objeto', 'geometry']], how='union').explode().reset_index(drop=True)
df_redvial_lm = df_redvial_lm.loc[df_redvial_lm.objeto.notnull()].reset_index()
df_redcol_lm = gpd.overlay(df_redcol, df_muni[['objeto','geometry']], how='union').explode().reset_index(drop=True)
df_redcol_lm = df_redcol_lm.loc[df_redcol_lm.objeto.notnull()].reset_index()
df_estcol_lm = df_estcol.sjoin(df_muni[['geometry']])
df_redffcc_lm = gpd.overlay(df_redffcc, df_muni[['objeto','geometry']], how='union').explode().reset_index(drop=True)
df_redffcc_lm = df_redffcc_lm.loc[df_redffcc_lm.objeto.notnull()].reset_index()
df_estffcc_lm = df_estffcc.sjoin(df_muni[['geometry']])

# save layer
df_estcol_lm.drop(columns='index_right', inplace=True)
df_estffcc_lm.drop(columns='index_right', inplace=True)
df_redvial_lm.to_file("../DatosProcesados/Urbanos/RedVial.geojson", driver='GeoJSON')
df_redcol_lm.to_file("../DatosProcesados/Urbanos/RecorridoColectivos.geojson", driver='GeoJSON')
df_estcol_lm.to_file("../DatosProcesados/Urbanos/EstacionesColectivos.geojson", driver='GeoJSON')
df_redffcc_lm.to_file("../DatosProcesados/Urbanos/RedFerrocarril.geojson", driver='GeoJSON')
df_estffcc_lm.to_file("../DatosProcesados/Urbanos/EstacionesFerrocarril.geojson", driver='GeoJSON')

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:11: UserWarning: `keep_geom_type=True` in overlay resulted in 1 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  # This is added back by InteractiveShellApp.init_path()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  # This is added back by InteractiveShellApp.init_path()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:13: UserWarning: `keep_geom_type=True` in overlay resulted in 1 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  del sys.path[0]
c:\Users\Compumar\anaconda3\envs

### Shops

In [15]:
# import data
df_shop = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/OSM_data/OSM_shop.geojson")
df_resto = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/OSM_data/OSM_amenity_restaurant.geojson")
df_ffood = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/OSM_data/OSM_amenity_fast_food.geojson")
df_cafe = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/OSM_data/OSM_amenity_cafe.geojson")
df_bar = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/OSM_data/OSM_amenity_bar.geojson")
df_bank = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/OSM_data/OSM_amenity_bank.geojson")
df_atm = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/OSM_data/OSM_amenity_atm.geojson")
# concat data
df_food = pd.concat([df_resto, df_ffood, df_cafe, df_bar]).reset_index(drop=True)
df_econ = pd.concat([df_bank, df_atm]).reset_index(drop=True)
# filter fields
df_shop =df_shop[['full_id', 'name', 'shop', 'geometry']]
df_food =df_food[['full_id', 'name', 'amenity', 'geometry']]
df_econ =df_econ[['full_id', 'name', 'amenity', 'brand', 'geometry']]

# buildings in La Matanza
df_shop_lm = df_shop.sjoin(df_muni[['geometry']])
df_food_lm = df_food.sjoin(df_muni[['geometry']])
df_econ_lm = df_econ.sjoin(df_muni[['geometry']])

# save layer
df_shop_lm.drop(columns='index_right', inplace=True)
df_food_lm.drop(columns='index_right', inplace=True)
df_econ_lm.drop(columns='index_right', inplace=True)
df_shop_lm.to_file("../DatosProcesados/Urbanos/LocalesComercios.geojson", driver='GeoJSON')
df_food_lm.to_file("../DatosProcesados/Urbanos/LocalesComida.geojson", driver='GeoJSON')
df_econ_lm.to_file("../DatosProcesados/Urbanos/BancosCajeros.geojson", driver='GeoJSON')

### Land Use

In [ ]:
# import data
df_usosuelo = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/Uso de suelo/uso_de_suelo.shp")
# remove null geometry
df_usosuelo = df_usosuelo.loc[df_usosuelo.geometry.notnull()]

# overlay with La Matanza
df_usosuelo_lm = gpd.overlay(df_usosuelo, df_muni[['objeto','geometry']], how='intersection')

# calculate surface
df_usosuelo_lm = df_usosuelo_lm.to_crs("EPSG:5347")
df_usosuelo_lm['sup_m2'] = df_usosuelo_lm.area
df_usosuelo_lm = df_usosuelo_lm.to_crs("EPSG:4326")

# save layer
df_usosuelo_lm.to_file("../DatosProcesados/Urbanos/UsosDeSuelo.geojson", driver='GeoJSON')

### Security

In [16]:
# import data
df_com = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/comisarias/shapefile/comisarias.shp")
df_commujer = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/comisarias-de-la-mujer/shapefile/comisarias-de-la-mujer.shp")
df_asisvic = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Urbano/centros-de-asistencia-a-victimas/shapefile/centros-de-asistencia-a-victimas.shp")
# concat dfs
df_secur = pd.concat([df_com, df_commujer, df_asisvic]).reset_index(drop=True)

# buildings in La Matanza
df_secur_lm = df_secur.sjoin(df_muni[['geometry']])

# save layer
df_secur_lm.drop(columns='index_right', inplace=True)
df_secur_lm.to_file("../DatosProcesados/Urbanos/EstablecimientosSeguridad.geojson", driver='GeoJSON')

# Socialdemographic Data

### Census Data

In [ ]:
# import data
df_pobla = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Sociodemografico/PBA_poblacion_y_datos_basicos_2010.shp")
df_nbi = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/Sociodemografico/PBA_nbi_2010.shp")
# rename nbi fields
df_nbi.rename(columns={'personas_c':'nbi_pers', 'personas_1':'nbi_pers_porc', 'hogares_co':'nbi_hog', 'hogares__1':'nbi_hog_porc'}, inplace=True)
# merge both dfs
df_census = df_pobla.merge(df_nbi[['link', 'nbi_pers', 'nbi_pers_porc', 'nbi_hog', 'nbi_hog_porc']], on='link', how='inner')

# filter ppolygons in La Matanza
df_census['filter_lm'] = df_census.link.str[:5]
df_census_lm = df_census.loc[df_census['filter_lm'] == '06427']
df_census_lm.drop(columns='filter_lm', inplace=True)

# calculate surface and density
df_census_lm = df_census_lm.to_crs("EPSG:5347")
df_census_lm['sup_m2'] = df_census_lm.area
df_census_lm = df_census_lm.to_crs("EPSG:4326")
df_census_lm['densidad_habm2'] = df_census_lm['personas'] /df_census_lm['sup_m2']
# order fields
df_census_lm = df_census_lm[['link', 'varon', 'mujer', 'personas',  'nbi_pers', 'nbi_pers_porc', 'hogares', 'viv_part',
       'viv_part_h', 'nbi_hog', 'nbi_hog_porc', 'densidad', 'densidad_habm2', 'sup_m2', 'geometry']]

# save layer
df_census_lm.to_file("../DatosProcesados/Sociodemograficos/DatosCensales.geojson", driver='GeoJSON')

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\pandas\core\frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## Sex and Age data

In [88]:
df_agesex = pd.read_csv("D:/JOB/UNSAM/2022 - CII/Datos/redatam/poblacion_por_sexo_y_edad.csv", encoding='latin', header=None)
df_agesex = df_agesex.iloc[:, :4]
df_agesex.columns = ['range', 'varon', 'mujer', 'total']
df_agesex['radio_link'] = '0'
df_agesex = df_agesex[['radio_link', 'range', 'varon', 'mujer', 'total']]
df_agesex = df_agesex.dropna(subset = ['range']).reset_index(drop=True)
df_agesex.loc[df_agesex.range == '5-sep', 'range'] = str("5-9")
df_agesex.loc[df_agesex.range == 'oct-14', 'range'] = str("10-14")
df_agesex = df_agesex.iloc[:28616]
df_agesex.head()

,radio_link,range,varon,mujer,total
0,0,AREA # 064270101,6458,NaN,NaN
1,0,Edades quinquenales,Sexo,NaN,NaN
2,0,0-4,14,5,19
3,0,5-9,14,9,23
4,0,10-14,5,7,12


In [89]:
radio_link_value = '0'
for i in range(len(df_agesex)):
    if len(list(df_agesex.loc[i, 'range'].split(' # '))) == 2:
        radio_link_value = df_agesex.loc[i, 'range'].split(' # ')[1]
        df_agesex.loc[i, 'radio_link'] = radio_link_value
    else:
        df_agesex.loc[i, 'radio_link'] = radio_link_value


In [90]:
df_agesex = df_agesex.loc[(df_agesex.range == '0-4') | (df_agesex.range == '5-9') | (df_agesex.range == '10-14') | (df_agesex.range == '15-19') | (df_agesex.range == '20-24')
            | (df_agesex.range == '25-29') | (df_agesex.range == '30-34') | (df_agesex.range == '35-39') | (df_agesex.range == '40-44') | (df_agesex.range == '45-49')
            | (df_agesex.range == '50-54') | (df_agesex.range == '55-59') | (df_agesex.range == '60-64') | (df_agesex.range == '65-69') | (df_agesex.range == '70-74')
            | (df_agesex.range == '75-79') | (df_agesex.range == '80-84') | (df_agesex.range == '85-89') | (df_agesex.range == '90-94') | (df_agesex.range == '95 y más')]
df_agesex.reset_index(inplace=True, drop=True)
df_agesex.loc[df_agesex.varon == '-', 'varon'] = 0
df_agesex.loc[df_agesex.mujer == '-', 'mujer'] = 0
df_agesex.loc[df_agesex.total == '-', 'total'] = 0
df_agesex.head(25)

,radio_link,range,varon,mujer,total
0,064270101,0-4,14,5,19
1,064270101,5-9,14,9,23
2,064270101,10-14,5,7,12
3,064270101,15-19,7,7,14
4,064270101,20-24,6,11,17
5,064270101,25-29,13,18,31
6,064270101,30-34,19,31,50
7,064270101,35-39,19,22,41
8,064270101,40-44,15,11,26
9,064270101,45-49,10,11,21


In [98]:
df_agesex.to_csv("../DatosProcesados/redatam/LM_pob_sexage.csv")

In [76]:
df_agesex

,radio_link,range,varon,mujer,total
0,064270101,0-4,14,5,19
1,064270101,5-9,14,9,23
2,064270101,10-14,5,7,12
3,064270101,15-19,7,7,14
4,064270101,20-24,6,11,17
...,...,...,...,...,...
24725,064279724,75-79,11.453,17.022,28.475
24726,064279724,80-84,6.533,11.865,18.398
24727,064279724,85-89,2.941,6.263,9.204
24728,064279724,90-94,739,2.084,2.823
